In [1]:
import sys
sys.path.append('/workspace/Documents')
# imports
import os, sys
import numpy as np 
import pandas as pd
import random
import nibabel as nb
import cv2
import ast
import matplotlib.pyplot as plt
from scipy.ndimage import zoom
from skimage.measure import block_reduce

import Diffusion_motion_field.Build_lists.Build_list as Build_list
import Diffusion_motion_field.functions_collection as ff
import Diffusion_motion_field.Data_processing as Data_processing

main_path = '/mnt/camca_NAS/4DCT/pictures/manuscript'

## image

In [ ]:
save_folder = os.path.join(main_path, 'illustration_of_MVF')
ff.make_folder([save_folder])
patient_class = 'Pull_2'
patient_id = 'case_72'
mvf = nb.load(os.path.join('/mnt/camca_NAS/4DCT/mvf_warp0_onecase', patient_class, patient_id, 'voxel_final/0.nii.gz')).get_fdata()
print(mvf.shape)

In [ ]:
# plot x
slice_n = 48
mvf_x = mvf[:,:,slice_n,0]; mvf_x = Data_processing.crop_or_pad(mvf_x, [100,100], np.min(mvf_x))

mvf_x = block_reduce(image=mvf_x, block_size=(4,4), func=np.mean)
# mvf_x = np.zeros_like(mvf_x)

plt.figure(figsize=(5, 5))
im = plt.imshow(mvf_x.T, cmap='hot', origin='lower' ,vmin = -5, vmax = 5)  # 'jet' or 'viridis' are common
plt.axis('off')
# plt.colorbar(im, label='Displacement (voxel)')
# plt.title('X-direction Motion Vector Field')
# plt.xlabel('X')
# plt.ylabel('Y')
plt.tight_layout()
# plt.show()
plt.savefig(os.path.join(save_folder, 'mvf_t0_downsampled.pdf'),format = 'pdf', bbox_inches='tight',pad_inches=0)



In [ ]:
# show that downsampled MVF can faithfully represent original MVF
# image_warped_mvf_original is by applying original MVF
# image_warped_mvf_down_up is by applying downsampled-then-upsample MVF
slice_n = 48
WL = -220
WW = 1770
img_ori = nb.load(os.path.join('/mnt/camca_NAS/4DCT/reference/',patient_class, patient_id, 'image_warped_mvf_original/6.nii.gz')).get_fdata()[:,:,slice_n]
img_ori = Data_processing.crop_or_pad(img_ori, [100,100], np.min(img_ori))

img_down_up  = nb.load(os.path.join('/mnt/camca_NAS/4DCT/reference/',patient_class, patient_id, 'image_warped_mvf_down_up/6.nii.gz')).get_fdata()[:,:,slice_n]
img_down_up = Data_processing.crop_or_pad(img_down_up, [100,100], np.min(img_down_up))
ff.save_grayscale_image(np.flip(img_ori.T,axis = 0),os.path.join(save_folder,'img_warped_mvf_original.png'), WL=WL, WW=WW)
ff.save_grayscale_image(np.flip(img_down_up.T,axis = 0),os.path.join(save_folder,'img_warped_mvf_down_up.png'), WL=WL, WW=WW)

img_diff = abs(img_down_up - img_ori)
mae = np.mean(img_diff)
print('mae:', mae)
ff.save_grayscale_image(np.flip(img_diff.T,axis = 0),os.path.join(save_folder,'img_diff.png'), WL=WL, WW=WW)

## statistics

In [3]:
data_sheet = os.path.join('/mnt/camca_NAS/4DCT/Patient_lists/mgh/patient_list_MVF_diffusion_train_test.xlsx')
b = Build_list.Build(data_sheet)
patient_class_list, patient_id_list,_ = b.__build__(batch_list = [0,1,2,3,4,5])

results = []
for i in range(0,patient_class_list.shape[0]):
    patient_class = patient_class_list[i]
    patient_id = patient_id_list[i]
    
    if os.path.isfile(os.path.join('/mnt/camca_NAS/4DCT/reference/', patient_class, patient_id, 'image_warped_mvf_original/6.nii.gz')) is False:
        print('skipped:', patient_class, patient_id)
        continue
    image_by_original_mvf = nb.load(os.path.join('/mnt/camca_NAS/4DCT/reference/', patient_class, patient_id, 'image_warped_mvf_original/6.nii.gz')).get_fdata()
    image_by_down_up_mvf = nb.load(os.path.join('/mnt/camca_NAS/4DCT/reference/', patient_class, patient_id, 'image_warped_mvf_down_up/6.nii.gz')).get_fdata()

    mae = np.mean(np.abs(image_by_down_up_mvf - image_by_original_mvf))
    results.append([patient_class, patient_id, mae])
    df = pd.DataFrame(results, columns=['patient_class', 'patient_id', 'mae'])
    df.to_excel(os.path.join('/mnt/camca_NAS/4DCT/Patient_lists/mgh/mae_MVF_down_up_vs_original.xlsx'), index=False)



skipped: Pull_3 case_128
skipped: Pull_2 case_85
